In [1]:
import tracemalloc

# Load Database

In [2]:
path = "datasets/"
datasets = [
    "accidents_utility.txt",        # 0
    "chainstore_utility.txt",       # 1
    "chess_utility.txt",            # 2
    "connect_utility.txt",          # 3
    "kosarak_utility.txt",          # 4
    "mushroom_utility.txt"          # 5
]

In [3]:
database = []
trans = []
dtb_idx = 2
with open(path + datasets[dtb_idx], "r") as reader:
    lines = reader.readlines()
    for line in lines:
        line = line.split(":")
        items = [int(it) for it in line[0].split(" ")]
        uT = int(line[1])
        itemU = [int(u) for u in line[2].split(" ")]
        trans = [items, uT, itemU]
        database.append(trans)

In [4]:
len(database)

3196

# Preliminaries

In [5]:
from collections import defaultdict

In [6]:
distinct_items = []
U = defaultdict()
TWU = defaultdict()
SUP = defaultdict()

for transaction in database:
    for i, item in enumerate(transaction[0]):
        if item in distinct_items:
            U[item] += transaction[2][i]
            TWU[item] += transaction[1]
            SUP[item] += 1  
        else:
            distinct_items.append(item)   
            U[item] = transaction[2][i]
            TWU[item] = transaction[1]
            SUP[item] = 1  

In [7]:
ikeep = [] 
minCor = 0.86
minUtil = 600000

# SearchCoHUI

In [8]:
def search_cohui(itemset_x, ux, rux, db_project_x, k):
     '''
     @func: SearchCoHUI
     @param: 
     - itemset_x: prefix itemset
     - U(X): utility of X
     - RU(X): the remain utility of X
     - db_project_x : projected database with X prefix
     - k: length of items set X.
     @return:  CoHUIs
     '''
     cohuis = []
     global ikeep
     # 1. for i = k to |I keep| do 
     for i in range(k, len(ikeep)):
          # 2. LastItem = Ikeep[i]
          # LastItem is an item that is extended from X itemset.
          last_item = ikeep[i]

          #  3.  X' =  X joins LastItem; X': extended from X by 'LastItem'
          x_extend = itemset_x.copy()
          x_extend.append(last_item)
          #  4. Initial U(X') = U(X); RU(X') = 0;
          #  SUP(X') = 0; ULA = U(X) + RU(X)
          ux_extend = ux
          rux_extend = 0
          supx_extend = 0
          ULA = ux + rux
          
          #  5. for T in project database 
          db_project_xtend = []
          for trans in db_project_x:
               # 6. set j = 0, xj in T
               j = 0
               u_temp = trans[1]
               # utemp = uT
               if last_item not in trans[0]:
                    continue
               # 8. while (j < T.len AND lastItem is after xj)
               while j < len(trans[0]) and \
               trans[0].index(trans[0][j]) < trans[0].index(last_item):
                    # 9. decrement uTemp by xj util
                    u_temp -= trans[2][j]
                    # incre j by 1
                    j+=1

               # 12 if (j == |T| OR last item is after x_j)
               if j == len(trans[0]) or trans[0].index(trans[0][j]) > trans[0].index(last_item):
                    # 13. Decrement U(X') by pru(T);
                    ux_extend -= trans[3]
                    # 14. Decrement ULA by (pru(T) + uT(T))
                    ULA -= (trans[3] + trans[1])
                    # 15. if ULA < minUtil then return;//LA-Prune
                    if ULA < minUtil:
                         # 16. Continue; if Xextend is not in T, then decrease ULA
                         # once ULA < minUtil stop the the projection with Xtend with return
                         break # TODO continue next item in ikeep

               else: # 17. else 
                    # 18 Increment U(X') by u(xj; T)
                    ux_extend += trans[2][j]
                    # 19 Increment SUP(X') by 1;
                    supx_extend += 1
                    # 20 if j < |T| then
                    if j < len(trans[0]):
                         # Calculate projected transaction with X'
                         # 21. T\X' = T.Get(j + 1, |T|)
                         # 22. pru(T\X') = (pru(T) + u(xj,T))
                         # 23. uT(T\X') = uTemp
                         new_trans = [
                              trans[0][j+1:], # 21
                              u_temp, # 22
                              trans[2][j+1:], 
                              trans[3] + trans[0][j]# 22
                         ]
                         # 24. dbProjectX' :add(newTran)
                         db_project_xtend.append(new_trans)
                         # 25. Increment RU(X') by uTemp;
                         rux_extend += u_temp

          
          if supx_extend > 0:
               kulc = (1/len(x_extend))*sum([supx_extend/SUP[it] for it in x_extend])
               if kulc >= minCor:
                    if ux_extend >= minUtil:                       
                         cohuis.append(x_extend + [[kulc], sum([U[it] for it in x_extend])])
               
                    if (ux_extend + rux_extend > minUtil):
                         cohuis += search_cohui(itemset_x=x_extend, ux=ux_extend, rux=ux_extend, db_project_x=db_project_xtend, k=k+1)
          
     return cohuis
    

# CoHUI-Miner

In [9]:
def cohui_miner(database, min_corr, min_util):
    '''
    @function: runCoHUIMiner: this function is used to run the correlated high-utility itemsets-Miner
         in which has some improvements comparing to its predecessor
    @param D transactional database
    @param minCor user-specified minimum correlation threshold
    @param minUtil user-specified minimum utility threshold
    @return: set of all correlated HUIs.
    '''
    cohuis = []
    global ikeep
    # 1. scan database for SUP, TWU, U

    # 2. construct I_keep = {i in Ikeep| TWU(i) >= minUtil}
    for item in distinct_items:
        if TWU[item] > min_util:
            ikeep.append(item) 
        else:
            # 3.update SUP, U in database 
            # (1) SUP = {SUP(i) | i in IKeep}
            # (2) U(i) = {U(i) | i in IKeep}    
            SUP.pop(item)
            U.pop(item)

    # 3.update SUP, U in database 
    # (3): Eliminate i from database D if i not in Ikeep    
    for trans in database:
        rmv_idx = []
        for idx in range(len(trans[0])):
            if trans[0][idx] in ikeep:
                continue
            else:
                rmv_idx += [idx]
        rmv_idx.sort(reverse=True)
        for idx in rmv_idx:
            trans[0].pop(idx)
            trans[2].pop(idx)
    
    # 4.1 sort Ikeep in the increasing order of SUP and 
    # 4.2 sort items of all transactions in D with respect to I keep 
    ikeep = sorted(ikeep, key=lambda x: SUP[x])
    for trans in database:
        itu = sorted(zip(trans[0], trans[2]), key= lambda x: SUP[x[0]])    
        trans[0] = [it for it,_ in itu]
        trans[2] = [u for _,u in itu]
    
    #  5. for each item X in Ikeep do
    for item_x in ikeep:
        db_project_x = []
        # 6. if U(X) > minUtil then 
        if U[item_x] > min_util:
            # 7. add X to CoHUIs
            cohuis.append(set(item_x))
        #  8. end if
        #  9. set RU(X) = 0
        ru_x = 0
        # 10. for each T in D do
        for trans in database:
            # 11. set j = 0; x_j in T; uTemp = u(T)
            j = 0
            u_temp = trans[1]

            # 12. while (j < |T| AND item X is after x_j) do
            if item_x not in trans[0]:
                continue # a quick check to process this transaction or not

            while j < len(trans[0]) and \
            trans[0].index(trans[0][j]) < trans[0].index(item_x):
                # 13. decrement uTemp by u(xj, T)
                u_temp -= trans[2][j]
                #  14. increment j by 1
                j += 1
                #  15. end while

            # 16. if (j == len(T.items) OR item x_j is after item X) 
            # then continue
            if j == len(trans[0]) or trans[0].index(trans[0][j]) > trans[0].index(item_x):
                continue

            # 17. else if (j < len(T.items)) then calculate projected transaction
            if j < len(trans[0]):
                # 18. initial T\X = T.Get(j + 1, |T|)
                # 19. PRU(T\X) = u(xj,T)
                # 20. uT(T\X) = uTemp
                new_trans = [
                    trans[0][j+1:], # 18
                    u_temp, # 20
                    trans[2][j+1:], 
                    trans[0][j] # 19
                ]
                # 21. dbProjectX.add(newTran)
                db_project_x.append(new_trans)
                #  22. Increment RU(X) by uTemp
                ru_x += u_temp
                # 23. end if 
            # 24. end for
        # 25. SearchCoHUI(X, U(X), RU(X), dbProjectX, 1)
        cohuis += search_cohui(itemset_x=[item_x], ux=U[item_x], rux=ru_x, db_project_x=db_project_x, k=1)
        # 26. end for
    return cohuis

# main

In [10]:
tracemalloc.start()
result = cohui_miner(database=database, min_corr=minCor, min_util=minUtil)
first_peak, first_size = tracemalloc.get_traced_memory()

In [11]:
with open("pyout.txt", "w") as writer:
    writer.write(f"database {datasets[dtb_idx]}\n")
    writer.write(f"cohui_miner [min_corr: {minCor}, min_util: {minUtil}]\n")
    writer.write(f"no. cohuis found: {len(result)}\n")
    for cohui in result:
        writer.write(cohui.__str__() + "\n")

In [12]:
BYTE_PER_MEGABYTE = 1024 * 1024
mem = (first_peak - first_size) / BYTE_PER_MEGABYTE
print(f"MEMORY USED IN MB: {mem:.2f}")

MEMORY USED IN MB: -7.29
